# MCSCF

There are many cases where the electronic structure of the system is not well described by a single determinant. This can occur due to covalent bond-breaking, in excited states, or even in the ground state equilibrium structure of some molecules, in particular metal complexes. In these cases, all single reference methods like Hartree-Fock, DFT, MP2, etc... may give unreliable results, and it is recommended to turn to multi-configurational and multi-reference methods.

VeloxChem is mostly focused on Hartree-Fock and DFT properties, but some libraries exist to complement these functionalities, for example Gator for ADC or Penguin for coupled cluster and perturbation theory. Here we are going to look at MultiPsi, which extends seamlessly VeloxChem for multi-configurational and multi-reference calculations.

In [ ]:
import veloxchem as vlx
import multipsi as mtp

## Ground state of the ozone molecule

In [ ]:
mol_str = """
O 0.0000   0.0000   0.0000
O 0.0000   1.0885   0.6697
O 0.0000  -1.0885   0.6697
"""

molecule = vlx.Molecule.read_str(mol_str)
basis = vlx.MolecularBasis.read(molecule,"def2-svp")

molecule.show()

Before running MCSCF, we need starting orbitals. We can run a Hartree-Fock or DFT calculation using VeloxChem or simply use an inexpensive guess.

In [ ]:
orbguess = mtp.OrbitalGuess()
start_orbs = orbguess.compute(molecule, basis)

We then need to define an active space. Here, we will use all valence 2p orbitals, i.e. 9 orbitals. This corresponds to 12 electrons. With this, we have all we need to run a MCSCF.

In [ ]:
orb_space = mtp.OrbSpace(molecule, start_orbs)
orb_space.cas(12, 9)

mcscf_drv = mtp.McscfDriver()
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space)

It is good to always check the orbitals after a calculation. The orb_space object contains the updated orbitals with the active space information.

In [ ]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, orb_space)

The orbitals in the active space are sorted by occupation numbers, not by energy (although here they match). The HOMO and LUMO in the ozone molecule have occupations differing significantly from 2 and 0, highlighting that this molecule is strongly correlated and would not be well described by single reference methods like DFT.

## UV-spectrum of benzene
MCSCF is particularly useful to compute excited properties. There are several ways to compute excited states with MCSCF, but a very simple one is using state-averaging, i.e. computing all states simultaneously and using the same orbitals.

Here we use benzene as a simple example.

In [ ]:
mol_str = """
C  1.400   0.000  0.000
C  0.700   1.212  0.000
C -0.700   1.212  0.000
C -1.400   0.000  0.000
C -0.700  -1.212  0.000
C  0.700  -1.212  0.000
H  2.480   0.000  0.000
H  1.240   2.148  0.000
H -1.240   2.148  0.000
H -2.480   0.000  0.000
H -1.240  -2.148  0.000
H  1.240  -2.148  0.000
"""
molecule = vlx.Molecule.read_str(mol_str)
basis = vlx.MolecularBasis.read(molecule, 'def2-svp')
molecule.show()

In [ ]:
orbguess = mtp.OrbitalGuess()
start_orbs = orbguess.compute(molecule, basis)

orb_space = mtp.OrbSpace(molecule, start_orbs)
orb_space.cas_orbitals([12, 17, 18, 21, 22, 23]) #found through visualization

mcscf_drv = mtp.McscfDriver()
mcscf_drv.gradient_thresh = 1.0e-6
mcscf_drv.e_change_thresh = 1.0e-6
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space, 3)

Now that we have computed all states, we can compute transition properties using a state-interaction module. By default, it computes oscillator and rotatory strengths.

In [ ]:
state_interaction = mtp.StateInteraction()
si_results = state_interaction.compute(molecule, basis, mcscf_dict)

## MC-PDFT of a metal complex

The ozone example worked immediately as we used all valence orbitals and they were conveniently located around the HOMO-LUMO gap.

This is not always the case, and sometimes, we need manual selection. Here let's look at the metal complex chromyl chloride. We're also going to use MC-PDFT with the translated BLYP functional (tBLYP).

In [ ]:
mol_str = """
Cr             0.0000        -0.0000         0.0000
O              0.9204         0.0000         1.2782
O              0.9204         0.0000        -1.2782
F             -0.9817         1.4162        -0.0000
F             -0.9817        -1.4162        -0.0000
"""
molecule = vlx.Molecule.read_str(mol_str)
basis = vlx.MolecularBasis.read(molecule, 'def2-sv(p)')
molecule.show()

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "BLYP"
scf_drv.ri_coulomb = True
scf_results = scf_drv.compute(molecule, basis)

We ideally want the Cr - O orbitals, which form the most strongly correlated bonds, but the Cr-Cl and Cl lone pairs are close to the HOMO-LUMO gaps, so we need to manually choose them. The simplest is to use the orbital viewer from MultiPsi to select the orbitals visually and save them in a file.

In [ ]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, scf_drv.molecular_orbitals)

In [ ]:
# chosen orbitals 20 23 26 27 28 30 31 32 33 34
orb_space = mtp.OrbSpace(molecule, "input-cas.h5")

mcscf_drv = mtp.McscfDriver()
mcscf_drv.xcfun = "tBLYP"
mcscf_drv.ri = True
mcscf_drv.gradient_thresh = 1.0e-6
mcscf_drv.e_change_thresh = 1.0e-6
mcscf_drv.ci_extra_tightness = 1.0e-1
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space)

Again, we should visualize the final orbitals. Note that the occupation numbers are closer to 2 and 0 than in the ozone example, but this is expected when using MC-PDFT. For reference, the corresponding MCSCF occupation numbers are:

```
1.93714 1.89203 1.88566 1.86014 1.84713 0.17316 0.12314 0.11925 0.09872 0.06363
```

In [ ]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, orb_space)